In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
import sys
sys.path.append('../code')
import scrape, features
import dataframe_utilities as util

# Update DB & Get Today Games

In [4]:
last_day = pd.to_datetime(pd.read_csv('../data/game_summaries.csv')['date']).max()
get_day = last_day + pd.Timedelta(days=1)
get_day

Timestamp('2019-08-19 00:00:00')

In [5]:
while get_day.date() < pd.datetime.today().date():
    links = scrape.get_game_links(get_day)
    for l in links:
        scrape.process_link(l)
    get_day += + pd.Timedelta(days=1)

ANA201908180 done
ARI201908180 done
ATL201908180 done
BOS201908180 done
CIN201908180 done
COL201908180 done
KCA201908180 done
NYA201908180 done
OAK201908180 done
PHI201908180 done
PIT201908180 done
TBA201908180 done
TEX201908180 done
TOR201908180 done
WAS201908180 done


In [6]:
test_df = scrape.get_today_games()

no pitcher {'away_team_abbr': 'SEA', 'home_team_abbr': 'TBR', 'time': '7:10PM', 'away_pitcher': 'castidi01'}
no pitcher {'away_team_abbr': 'PHI', 'home_team_abbr': 'BOS', 'time': '7:10PM', 'away_pitcher': 'nolaaa01'}


In [7]:
test_df

,away_pitcher,away_team_abbr,home_pitcher,home_team_abbr,time,date
0,heanean01,LAA,palumjo01,TEX,2:05PM,2019-08-20
1,strasst01,WSN,archech01,PIT,7:05PM,2019-08-20
2,kellebr01,KCR,bundydy01,BAL,7:05PM,2019-08-20
3,biebesh01,CLE,matzst01,NYM,7:10PM,2019-08-20
4,castidi01,SEA,NaN,TBR,7:10PM,2019-08-20
5,quantca01,SDP,grayso01,CIN,7:10PM,2019-08-20
6,nolaaa01,PHI,NaN,BOS,7:10PM,2019-08-20
7,hernael01,MIA,keuchda01,ATL,7:20PM,2019-08-20
8,gonzagi01,MIL,wachami01,STL,7:45PM,2019-08-20
9,beedety01,SFG,hamelco01,CHC,8:05PM,2019-08-20


# Process Stats for Today's Games

## Merge test and train dfs

In [8]:
test_df['is_night_game'] = True
test_df['is_night_game'][test_df['time'].str[:1].astype('int')<5] = False
test_df['is_night_game'][test_df['time'].str[1:2].isin(['0','1'])] = True #for 10,11 PM games
test_df.drop(columns='time', inplace=True)

In [9]:
test_df['is_test'] = True
test_df['home_team_win']=np.nan
test_df['game_id'] = test_df.home_team_abbr + test_df.date.astype('str').str.replace('-','') + '0'
test_df.shape

(16, 9)

In [10]:
df = features.get_game_df()
df['is_test'] = False
df.shape

(24094, 9)

In [11]:
df = pd.concat([df,test_df])
df = df.sort_values(by='date').reset_index(drop=True)
df.shape

(24110, 9)

## Add Features

In [12]:
df = features.add_trueskill_ratings(df)
df = features.add_rest_durations(df)
df.shape

(24110, 18)

In [13]:
#datetime
date = pd.to_datetime(df['date'])
df['season'] = date.dt.year
df['month']=date.dt.month
df['week_num'] = date.dt.week
df['dow']=date.dt.weekday.astype('int')

In [14]:
df['dh_game_no'] = pd.to_numeric(df['game_id'].str[-1:],errors='coerce')
df['date'] = (pd.to_datetime(df['date']) - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s') #epoch time

### Add Stats

In [15]:
games = features.get_games()
batting = features.get_batting()
pitching = features.get_pitching()
pitchers = features.get_pitchers()

#### Rolling 10 Day Stats

In [16]:
b_stats = ['batting_avg','leverage_index_avg', 'onbase_perc', 'onbase_plus_slugging']
df = features.add_10RA_rolling(batting, df, b_stats, True, 'batting')

In [17]:
pitching['SO_batters_faced'] = pitching['SO'] / pitching['batters_faced']
pitching['H_batters_faced'] = pitching['H'] / pitching['batters_faced']
pitching['BB_batters_faced'] = pitching['BB'] / pitching['batters_faced']

# create rolling stat
b_stats = ['earned_run_avg','SO_batters_faced','H_batters_faced','BB_batters_faced']
df = features.add_10RA_rolling(pitching, df, b_stats, True, 'team_pitching')

In [18]:
pitchers['SO_batters_faced'] = pitchers['SO'] / pitchers['batters_faced']
pitchers['H_batters_faced'] = pitchers['H'] / pitchers['batters_faced']
pitchers['BB_batters_faced'] = pitchers['BB'] / pitchers['batters_faced']

# create rolling stat
b_stats = ['earned_run_avg','SO_batters_faced','H_batters_faced','BB_batters_faced']
df = features.add_10RA_rolling(pitchers, df, b_stats, False, 'pitcher')

#### Games Stats

In [19]:
df = features.game_stats(games,df)

In [20]:
df.shape

(24110, 82)

#### Season Stats

In [21]:
batting_stats = ['A', 'AB', 'BB', 'H', 'PA', 'PO', 'R', 'RBI', 'SO', 'batting_avg',
             'leverage_index_avg', 'onbase_perc', 'onbase_plus_slugging', 'pitches', 
             're24_bat', 'slugging_perc', 'strikes_total', 'wpa_bat', 'wpa_bat_neg', 
             'wpa_bat_pos']
df = features.add_season_rolling(batting, df, batting_stats, True,'batting')
df.shape

(24110, 222)

In [22]:
pitching_stats = ['BB', 'ER', 'H', 'HR', 'IP', 'R', 'SO', 'batters_faced',
               'earned_run_avg', 'game_score', 'inherited_runners',
               'inherited_score', 'inplay_fb_total', 'inplay_gb_total', 'inplay_ld',
               'inplay_unk', 'leverage_index_avg', 'pitches', 're24_def',
               'strikes_contact', 'strikes_looking', 'strikes_swinging',
               'strikes_total', 'wpa_def','SO_batters_faced','H_batters_faced',
                'BB_batters_faced']
df = features.add_season_rolling(pitching, df, pitching_stats, True,'team_pitching')
df.shape

(24110, 411)

In [23]:
df = features.add_season_rolling(pitchers, df, pitching_stats, False,'pitcher')
df.shape

(24110, 600)

## Cleanup

In [24]:
df = util.fix_na(df, False)

# Generate Predictions

In [25]:
X_test = df[df.is_test].drop(columns=['is_test'])
cols = pd.read_csv("../data/df_w_features.csv").columns
X_test = X_test[cols]

X_test.drop(columns=['home_team_win','game_id'], inplace=True)

In [26]:
import pickle
encoder, model = pickle.load(open('../data/encoder_model.pk','rb'))
encoder, model

(CatBoostEncoder(cols=['home_team_abbr', 'away_team_abbr', 'home_pitcher', 'away_pitcher', 'home_team_season', 'away_team_season'],
         drop_invariant=False, handle_missing='value',
         handle_unknown='value', random_state=13, return_df=True,
         sigma=None, verbose=0),
 XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
        colsample_bytree=0.45, gamma=0, learning_rate=0.03,
        max_delta_step=0, max_depth=27, min_child_weight=7.0, missing=nan,
        n_estimators=100, n_jobs=1, nthread=None, num_boost_round=100,
        objective='binary:logistic', random_state=13,
        reg_alpha=6.417040000000001, reg_lambda=1, scale_pos_weight=1,
        seed=None, silent=True, subsample=0.05))

In [27]:
X_test = encoder.transform(X_test)

test_df = test_df.sort_values(by=['date','game_id']).reset_index(drop=True)
pred_df = df[df.is_test][['away_pitcher', 'away_team_abbr', 'home_pitcher', 'home_team_abbr']]
pred_df['xgb_probability']= model.predict_proba(X_test)[:,1]
pred_df['xgb_winner']=pred_df.home_team_abbr
pred_df['xgb_winner'][~model.predict(X_test)]=pred_df.away_team_abbr

pred_df

,away_pitcher,away_team_abbr,home_pitcher,home_team_abbr,xgb_probability,xgb_winner
24094,lopezre01,CHW,pinedmi01,MIN,0.525883,MIN
24095,castidi01,SEA,Unknown,TBR,0.531755,TBR
24096,gonzagi01,MIL,wachami01,STL,0.498785,MIL
24097,strasst01,WSN,archech01,PIT,0.488138,WSN
24098,germado01,NYY,baileho02,OAK,0.468748,NYY
24099,biebesh01,CLE,matzst01,NYM,0.471647,CLE
24100,reidfse01,TOR,kershcl01,LAD,0.574231,LAD
24101,barrija01,LAA,,TEX,0.583371,TEX
24102,quantca01,SDP,grayso01,CIN,0.499282,SDP
24103,beedety01,SFG,hamelco01,CHC,0.560410,CHC
